In [1]:
script = '''contract get_fib_n(n:int){
    let last_fib_calculated : int ;

    entry get_fib(n: int){
        let temp:int = fib(n);
    }

    func fib(n: int) : int{
        if (n <= 1) {
            then{
                return 1;
            }
            else {
                
                let a: int = n - 1;
                let b: int = n - 2;
                return fib(a) + fib(b);
            }
        }
    }
    
    entry exec(){
        let a: int = 5;
        let t: int = fib(a);
    }
}'''

In [2]:
# script = '''
#     contract test_address_type(){
#         let temp: address;

#         entry test_entry(a: int, b:nat, c:string, d:bool, e:address){
#             temp = c;
#             let f: address = e;
#             let x: string = c + e;
#             let y: address = "tz1icNwg34VPyWJevGRBz56r19MiqYeJQaxU";
#             let z: address = "hola mundo";
#             let z: address = y + z;
#             let s: string = "hello";
#             let v: address = test_func(s);
#         }

#         func test_func(a: address): address{
#             return a;
#         }
#     }
#     '''

In [3]:
from visitors.string_rep_visitor import FormatVisitor
from utils import run_tzscript_ast_building_pipeline

ast = run_tzscript_ast_building_pipeline(script)
# format = FormatVisitor()
# print(format.visit(ast))

0 	 <program> -> .contractid(<op-param-list{<stat_program_list>}, {'$'}
	 S' -> .<program>, {'$'} 

1 	 <program> -> contract.id(<op-param-list{<stat_program_list>}, {'$'} 

2 	 <program> -> contractid.(<op-param-list{<stat_program_list>}, {'$'} 

3 	 <op-param-list -> .), {'{'}
	 <op-param-list -> .<param-list>), {'{'}
	 <param-list> -> .<param>, {',', ')'}
	 <param-list> -> .<param-list>,<param>, {',', ')'}
	 <program> -> contractid(.<op-param-list{<stat_program_list>}, {'$'}
	 <param> -> .id:type, {',', ')'} 

4 	 <op-param-list -> <param-list>.), {'{'}
	 <param-list> -> <param-list>.,<param>, {',', ')'} 

5 	 <op-param-list -> <param-list>)., {'{'} 

6 	 <param-list> -> <param-list>,.<param>, {',', ')'}
	 <param> -> .id:type, {',', ')'} 

7 	 <param-list> -> <param-list>,<param>., {',', ')'} 

8 	 <param> -> id.:type, {',', ')'} 

9 	 <param> -> id:.type, {',', ')'} 

10 	 <param> -> id:type., {',', ')'} 

11 	 <param-list> -> <param>., {',', ')'} 

12 	 <op-param-list -> )., {'{'}

In [4]:
from visitors.semantic_check_visitor import SemanticCheckerVisitor
semantic_checker = SemanticCheckerVisitor()
a = semantic_checker.visit(ast)


In [5]:
from visitors.code_runner import CodeRunnerVisitor
runner = CodeRunnerVisitor()
runner.visit(ast)


declare var in storage
Entry dec Node
Func declaration node
Entry dec Node
Calling exec []
scope {}
declare var
num 5
declare var
Calling fib [<parser.tzscript_ast.VariableNode object at 0x7f2214d4e2e0>]
var
{}
{'a': {'type': <parser.tzscript_types.TzScriptInt object at 0x7f224430a880>, 'value': 5}}
Attr
n {'type': <parser.tzscript_types.TzScriptInt object at 0x7f224430a880>, 'value': None}
5
scope {'n': {'type': <parser.tzscript_types.TzScriptInt object at 0x7f224430a880>, 'value': 5}}
if Node
var
{'n': {'type': <parser.tzscript_types.TzScriptInt object at 0x7f224430a880>, 'value': 5}}
num 1
5 <= 1
declare var
var
{}
{'n': {'type': <parser.tzscript_types.TzScriptInt object at 0x7f224430a880>, 'value': 5}}
num 1
5 - 1
declare var
var
{'a': {'type': <parser.tzscript_types.TzScriptInt object at 0x7f224430a880>, 'value': 4}}
{'n': {'type': <parser.tzscript_types.TzScriptInt object at 0x7f224430a880>, 'value': 5}}
num 2
5 - 2
plus
Calling fib [<parser.tzscript_ast.VariableNode object at 0x